In [3]:
import pandas as pd

# Load historical funding rate data
df = pd.read_csv("dataset/binance_btc_usdt_1_year_data_8h.csv")

# Parameters
min_funding_rate = 0.0001  # Minimum funding rate to consider arbitrage
trade_size = 0.01  # BTC per trade

# Function to check if arbitrage opportunity exists
def check_arbitrage_opportunity(df, min_funding_rate):
    return df[abs(df['funding_rate']) >= min_funding_rate]

# Function to determine hedge positions
def open_hedge_position(trade_size, funding_rate):
    if funding_rate > 0: 
        side_spot = 'buy'
        side_futures = 'sell'
    else:
        side_spot = 'sell'
        side_futures = 'buy'
    
    print(f"Opened hedge positions: {side_futures} futures and {side_spot} spot.")
    return side_spot, side_futures

# Function to close hedge positions
def close_positions(trade_size, funding_rate):
    if funding_rate > 0:
        side_spot = 'sell'
        side_futures = 'buy'
    else:
        side_spot = 'buy'
        side_futures = 'sell'
    
    print(f"Closed hedge positions: {side_spot} spot and {side_futures} futures.")
    return side_spot, side_futures

# Function to calculate total profit
def calculate_total_profit(trade_size, funding_rate, mark_price, entry_spot, close_spot, entry_futures, close_futures):
    """
    Calculates total funding arbitrage profit:
    - Funding fee profit
    - Spot position profit/loss
    - Futures position profit/loss
    """

    # Funding Profit (Positive if funding > 0, Negative if funding < 0)
    funding_profit = trade_size * funding_rate * mark_price

    # Determine trade direction based on funding rate
    if funding_rate > 0:
        spot_side_multiplier = 1     # Buying spot (long)
        futures_side_multiplier = -1  # Shorting futures
    else:
        spot_side_multiplier = -1     # Selling spot (short)
        futures_side_multiplier = 1   # Longing futures

    # Spot Profit/Loss
    spot_pnl = trade_size * (close_spot - entry_spot) * spot_side_multiplier

    # Futures Profit/Loss
    futures_pnl = trade_size * (entry_futures - close_futures) * futures_side_multiplier

    # Total Profit
    total_profit = funding_profit + spot_pnl + futures_pnl

    return total_profit

# Function to run the funding arbitrage bot
def funding_arbitrage_bot(df, trade_size, min_funding_rate):
    results = []

    # Filter for arbitrage opportunities
    arbitrage_df = check_arbitrage_opportunity(df, min_funding_rate)

    for index, row in arbitrage_df.iterrows():
        timestamp = row['timestamp']
        funding_rate = row['funding_rate']
        mark_price = row['close_futures']  # Mark price is usually close to futures price
        entry_spot = row['close_spot']
        entry_futures = row['close_futures']

        print(f"\n[Trade {index}] Arbitrage Opportunity Found! Funding Rate: {funding_rate}")

        # Open hedge positions
        side_spot, side_futures = open_hedge_position(trade_size, funding_rate)

        # Simulate trade closing (next row)
        if index + 1 < len(df):
            close_spot = df.loc[index + 1, 'close_spot']
            close_futures = df.loc[index + 1, 'close_futures']

            # Close positions
            close_positions(trade_size, funding_rate)

            # Calculate profit
            total_profit = calculate_total_profit(
                trade_size, funding_rate, mark_price, entry_spot, close_spot, entry_futures, close_futures
            )

            print(f"Trade Cycle Complete. Total Profit: ${total_profit:.2f}")

            # Log results
            results.append([timestamp, funding_rate, trade_size, entry_spot, close_spot, entry_futures, close_futures, total_profit])

    # Save results to CSV
    results_df = pd.DataFrame(results, columns=[
        'timestamp', 'funding_rate', 'trade_size', 'entry_spot', 'close_spot', 'entry_futures', 'close_futures', 'total_profit'
    ])
    results_df.to_csv("dataset/funding_arbitrage_results.csv", index=False)
    print("\n✅ All trades completed. Results saved to 'funding_arbitrage_results.csv'.")

# Run the bot
funding_arbitrage_bot(df, trade_size, min_funding_rate)


[Trade 0] Arbitrage Opportunity Found! Funding Rate: 0.0001
Opened hedge positions: sell futures and buy spot.
Closed hedge positions: sell spot and buy futures.
Trade Cycle Complete. Total Profit: $1.47

[Trade 1] Arbitrage Opportunity Found! Funding Rate: 0.0001
Opened hedge positions: sell futures and buy spot.
Closed hedge positions: sell spot and buy futures.
Trade Cycle Complete. Total Profit: $-25.17

[Trade 2] Arbitrage Opportunity Found! Funding Rate: 0.0001
Opened hedge positions: sell futures and buy spot.
Closed hedge positions: sell spot and buy futures.
Trade Cycle Complete. Total Profit: $19.20

[Trade 3] Arbitrage Opportunity Found! Funding Rate: 0.0001
Opened hedge positions: sell futures and buy spot.
Closed hedge positions: sell spot and buy futures.
Trade Cycle Complete. Total Profit: $2.93

[Trade 4] Arbitrage Opportunity Found! Funding Rate: 0.0001
Opened hedge positions: sell futures and buy spot.
Closed hedge positions: sell spot and buy futures.
Trade Cycle Co

Check profits

In [ ]:
result_df = pd.read_csv("old_funding_arbitrage_results.csv")

In [ ]:
result_df

,timestamp,funding_rate,trade_size,entry_spot,close_spot,entry_futures,close_futures,total_profit
0,2024-02-12 16:00:00,0.0001,0.01,49917.27,49983.09,49943.6,50019.9,1.471144
1,2024-02-13 00:00:00,0.0001,0.01,49983.09,48727.47,50019.9,48753.1,-25.174180
2,2024-02-13 08:00:00,0.0001,0.01,48727.47,49699.59,48753.1,49696.4,19.202953
3,2024-02-13 16:00:00,0.0001,0.01,49699.59,49838.93,49696.4,49845.0,2.929096
4,2024-02-14 00:00:00,0.0001,0.01,49838.93,51590.00,49845.0,51616.4,35.274545
...,...,...,...,...,...,...,...,...
584,2025-01-07 08:00:00,0.0001,0.01,97952.26,96954.61,97915.3,96920.3,-19.828585
585,2025-01-08 08:00:00,0.0001,0.01,95493.77,95060.61,95450.0,95014.1,-8.595150
586,2025-01-08 16:00:00,0.0001,0.01,95060.61,93221.59,95014.1,93176.4,-36.672186
587,2025-01-09 00:00:00,0.0001,0.01,93221.59,94244.02,93176.4,94195.2,20.505476


In [6]:
print(result_df['total_profit'].sum())

504.46451258598836
